The scope of ETL project is as follows:
We have NFL player data from Wikipedia , Kaggle and Data.World and would want information on players who made it Pro-Bowl from 2017 to 1997
1. So we get the players name(list) from Wikipedia for each year's probowl selection ( Check pages from 2017--1997)
2. Once we get the information for 30 years of data , we remove the duplicates
3. then we join the player name with information ( csv file and spreadsheet from Kaggle and Data.World) to get additional information on the player.
4. We clean the data
5. We save the data to a cloud database and expose it as REST API's..


In [1]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from sqlalchemy import create_engine
import sqlalchemy
import requests
#import bamboolib as bam

In [2]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
base_url = 'https://en.wikipedia.org/wiki/'
base_url_forplayer = 'https://en.wikipedia.org'
allproplayers_info = {}

In [4]:
# Visit the following URL
# Our scope is All-Pro-Team from 1997 to 2017
nflseason_year = 1987
while nflseason_year < 2018:
    url = base_url + str(nflseason_year)+'_All-Pro_Team'
    try:
        browser.visit(url)
        time.sleep(3)
        #getting the html content from the browser instance launched using splinter
        html=browser.html
    #scrape the web page using beautiful soup and lxml parser
        soup = BeautifulSoup(html, 'lxml')
        player_tableColl = soup.find_all('table', class_='wikitable')
        for table in player_tableColl:
            tr_coll = table.find_all('tr')
            for item in tr_coll:
                data = item.find_all(['th','td'])
                if data:
                    try:
                        player = data[1].a.text
                        player_info = data[1].a['href']
                        player1 = data[2].a.text
                        player1_info = data[1].a['href']
                        #check if these players exist in our players dictionary else append them
                        if player not in allproplayers_info:
                            allproplayers_info.update({player:player_info})
                        if player1 not in allproplayers_info:
                            allproplayers_info.update({player1:player_info1})
                    except Exception:pass
                #print("exception")
    except Exception:pass
    nflseason_year = nflseason_year + 1
    
#print the dictionary
print(allproplayers_info)
    

{'Eric Dickerson': '/wiki/Eric_Dickerson', 'Mark Bavaro': '/wiki/Mark_Bavaro', 'Gary Zimmerman': '/wiki/Gary_Zimmerman', 'Mike Munchak': '/wiki/Mike_Munchak', 'Dwight Stephenson': '/wiki/Dwight_Stephenson', 'Morten Andersen': '/wiki/Morten_Andersen', 'Jim Arnold': '/wiki/Jim_Arnold_(American_football)', 'Reggie White': '/wiki/Reggie_White', 'Michael Carter': '/wiki/Michael_Carter_(athlete)', 'Fredd Young': '/wiki/Fredd_Young', 'Carl Banks': '/wiki/Carl_Banks', 'Hanford Dixon': '/wiki/Hanford_Dixon', 'Joey Browner': '/wiki/Joey_Browner', 'Boomer Esiason': '/wiki/Boomer_Esiason', 'Anthony Muñoz': '/wiki/Anthony_Mu%C3%B1oz', 'Tom Newberry': '/wiki/Tom_Newberry', 'Jay Hilgenberg': '/wiki/Jay_Hilgenberg', 'Scott Norwood': '/wiki/Scott_Norwood', 'Tim Krumrie': '/wiki/Tim_Krumrie', 'Mike Singletary': '/wiki/Mike_Singletary', 'Cornelius Bennett': '/wiki/Cornelius_Bennett', 'Frank Minnifield': '/wiki/Frank_Minnifield', 'Joe Montana': '/wiki/Joe_Montana', 'Christian Okoye': '/wiki/Christian_Okoy

In [5]:
#now you have the dictionary of players you have to get all the values and scrape thru those sites
draft_info_year = []
for value in list(allproplayers_info.values()):
    try:
        tempurl = base_url_forplayer + str(value)
        browser.visit(tempurl)
        time.sleep(2)
        #getting the html content from the browser instance launched using splinter
        html=browser.html
        #scrape the web page using beautiful soup and lxml parser
        soup = BeautifulSoup(html, 'lxml')
        # find the player name
        player_tbl = soup.find("table", {"class": "infobox vcard"})
        #print(player_tbl)
        list_of_table_rows = player_tbl.findAll('tr')
        #print(list_of_table_rows)
        for item in list_of_table_rows:
                data = item.find_all(['th'],string="NFL Draft:")
                if data:
                    try:
                        draft_year = data[0].find_next_sibling('td').find('a')['title'][0:4]
                        #update the player information dictionary

                    except Exception:pass
        key_to_update = [k for k,v in allproplayers_info.items() if v == value]
        print(key_to_update)
        allproplayers_info.update({key_to_update[0]:draft_year})
    except Exception:pass
print(allproplayers_info)

['Eric Dickerson']
['Mark Bavaro']
['Gary Zimmerman']
['Mike Munchak']
['Dwight Stephenson']
['Morten Andersen']
['Jim Arnold']
['Reggie White']
['Michael Carter']
['Fredd Young']
['Carl Banks']
['Hanford Dixon']
['Joey Browner']
['Boomer Esiason']
['Anthony Muñoz']
['Tom Newberry']
['Jay Hilgenberg']
['Scott Norwood']
['Tim Krumrie']
['Mike Singletary']
['Cornelius Bennett']
['Frank Minnifield']
['Joe Montana']
['Christian Okoye']
['Jerry Rice']
['Keith Jackson']
['Sean Landeta']
['Rod Woodson']
['Chris Doleman']
['Keith Millard']
['Karl Mecklenburg']
['Tim Harris']
['Albert Lewis']
['David Fulcher']
['Thurman Thomas']
['Jim Lachey']
['Bruce Matthews']
['Kent Hull']
['Nick Lowery']
['Bruce Smith']
['Michael Dean Perry']
['Pepper Johnson']
['Derrick Thomas']
['Jim Kelly']
['Michael Irvin']
['Marv Cook']
['Steve Wisniewski']
['Jeff Gossett']
['Mel Gray']
['Jerome Brown']
['Sam Mills']
['Pat Swilling']
['Darrell Green']
['Ronnie Lott']
['Steve Young']
['Emmitt Smith']
['Sterling Sharpe']

In [6]:
df_allpro = pd.DataFrame.from_dict(allproplayers_info, orient="index").reset_index()
df_allpro.columns = ['playername', 'year']
print(df_allpro)

            playername  year
0       Eric Dickerson  1983
1          Mark Bavaro  1985
2       Gary Zimmerman  1985
3         Mike Munchak  1982
4    Dwight Stephenson  1980
..                 ...   ...
267    Calais Campbell  2008
268     Chandler Jones  2012
269       Jalen Ramsey  2016
270        Kevin Byard  2016
271        Darius Slay  2013

[272 rows x 2 columns]


In [7]:
df_nfldraft = pd.read_excel('Resources/NFL Combine Data.xlsx', sheet_name='nfl_draft')
df_nfldraft.drop(['Unnamed: 0','Unnamed: 34'],axis=1,inplace=True)
df_nfldraft["Year"] = pd.to_numeric(df_nfldraft["Year"])

df_allpro["year"] = pd.to_numeric(df_allpro["year"],errors='coerce')
df_allpro_draft = pd.merge(df_nfldraft, df_allpro, left_on=['Player','Year'], right_on=['playername','year'])
df_allpro_draft

,Player_Id,Year,Rnd,Pick,Tm,Player,HOF,Pos,Position Standard,First4AV,...,Rush_TDs,Rec,Rec_Yds,Rec_Tds,Tkl,Def_Int,Sk,College/Univ,playername,year
0,GurlTo01,2015,1,10,STL,Todd Gurley,No,RB,RB,8,...,14.0,47.0,390.0,0.0,NaN,NaN,NaN,Georgia,Todd Gurley,2015.0
1,CollLa00,2015,2,33,NYG,Landon Collins,No,SS,DB,5,...,NaN,NaN,NaN,NaN,146.0,6.0,3,Alabama,Landon Collins,2015.0
2,LockTy00,2015,3,69,SEA,Tyler Lockett,No,WR,WR,9,...,0.0,74.0,989.0,6.0,NaN,NaN,NaN,Kansas St.,Tyler Lockett,2015.0
3,JohnDa08,2015,3,86,ARI,David Johnson,No,RB,RB,9,...,18.0,83.0,967.0,6.0,NaN,NaN,NaN,Northern Iowa,David Johnson,2015.0
4,MackKh00,2014,1,5,OAK,Khalil Mack,No,LB,LB,21,...,NaN,NaN,NaN,NaN,148.0,NaN,27,Buffalo,Khalil Mack,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,DoleCh00,1985,1,4,MIN,Chris Doleman,No,DE,DE,54,...,NaN,NaN,NaN,NaN,8.0,150.5,Pittsburgh,NaN,Chris Doleman,1985.0
189,LachJi00,1985,1,12,SDG,Jim Lachey,No,T,T,46,...,NaN,NaN,NaN,NaN,NaN,NaN,Ohio St.,NaN,Jim Lachey,1985.0
190,RiceJe00,1985,1,16,SFO,Jerry Rice,No,WR,WR,69,...,10.0,1549.0,22895.0,197.0,NaN,NaN,Miss. Valley St.,NaN,Jerry Rice,1985.0
191,BavaMa00,1985,4,100,NYG,Mark Bavaro,No,TE,TE,32,...,NaN,351.0,4733.0,39.0,NaN,NaN,Notre Dame,NaN,Mark Bavaro,1985.0


In [8]:
df_combine = pd.read_excel('Resources/NFL Combine Data.xlsx', sheet_name='Combine Results')
df_combine = df_combine[df_combine['Name'].isin(allproplayers_info)]
df_combine.shape
df_combine

,Year,Name,College,POS,Height (in),Weight (lbs),Hand Size (in),Arm Length (in),Wonderlic,40 Yard,Bench Press,Vert Leap (in),Broad Jump (in),Shuttle,3Cone,60Yd Shuttle
291,2017,Budda Baker,Washington,FS,69.63,195,9.00,30.75,14.0,4.45,15.0,32.5,115.0,4.08,6.76,NaN
658,2016,Jack Conklin,Michigan State,OT,77.75,308,10.38,35.00,NaN,5.00,25.0,30.0,103.0,4.57,7.63,NaN
661,2016,Pharoh Cooper,South Carolina,WR,71.13,203,9.13,32.25,NaN,NaN,15.0,31.0,115.0,NaN,NaN,NaN
685,2016,Ezekiel Elliott,Ohio State,RB,71.75,225,10.25,31.25,NaN,4.47,NaN,32.5,118.0,NaN,NaN,NaN
831,2016,Jalen Ramsey,Florida State,FS,73.25,209,9.50,33.38,NaN,4.41,14.0,41.5,135.0,4.18,6.94,11.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1987,Jerome Brown,Miami (FL),DT,74.30,292,9.75,33.00,NaN,4.86,17.0,NaN,NaN,NaN,NaN,NaN
10034,1987,Rich Gannon,Delaware,QB,74.30,193,9.50,30.00,NaN,4.58,7.0,32.5,110.0,4.11,NaN,NaN
10134,1987,Christian Okoye,Azusa Pacific (CA),RB,73.60,250,9.75,33.00,NaN,NaN,21.0,31.0,NaN,NaN,NaN,NaN
10180,1987,Steve Smith,Penn State,RB,72.90,244,10.75,31.00,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN


In [9]:
df_NFLChampsMerged = pd.merge(df_combine, df_allpro_draft, left_on=['Name','Year'], right_on=['Player','Year'])


# Remove Duplicate Chad Brown from 1993
df_NFLChampsMerged = df_NFLChampsMerged[df_NFLChampsMerged.Player_Id != 'BrowCh20']
df_NFLChampsMerged = df_NFLChampsMerged[df_NFLChampsMerged.Player_Id != 'BrowCh04']

df_NFLChampsMerged


,Year,Name,College,POS,Height (in),Weight (lbs),Hand Size (in),Arm Length (in),Wonderlic,40 Yard,...,Rush_TDs,Rec,Rec_Yds,Rec_Tds,Tkl,Def_Int,Sk,College/Univ,playername,year
0,2015,Landon Collins,Alabama,SS,72.00,228,9.38,31.50,17.0,4.53,...,NaN,NaN,NaN,NaN,146.0,6.0,3,Alabama,Landon Collins,2015.0
1,2015,Todd Gurley,Georgia,RB,72.63,222,10.00,31.50,12.0,4.52,...,14.0,47.0,390.0,0.0,NaN,NaN,NaN,Georgia,Todd Gurley,2015.0
2,2015,David Johnson,Northern Iowa,RB,72.63,224,9.63,31.25,NaN,4.50,...,18.0,83.0,967.0,6.0,NaN,NaN,NaN,Northern Iowa,David Johnson,2015.0
3,2015,Tyler Lockett,Kansas State,WR,69.88,182,8.38,30.00,NaN,4.40,...,0.0,74.0,989.0,6.0,NaN,NaN,NaN,Kansas St.,Tyler Lockett,2015.0
4,2014,Aaron Donald,Pittsburgh,DT,72.75,285,9.88,32.63,NaN,4.68,...,NaN,NaN,NaN,NaN,105.0,NaN,25,Pittsburgh,Aaron Donald,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,1987,Harris Barton,North Carolina,C,76.30,280,9.50,31.75,NaN,5.03,...,NaN,NaN,NaN,NaN,NaN,NaN,North Carolina,NaN,Harris Barton,1987.0
144,1987,Jerome Brown,Miami (FL),DT,74.30,292,9.75,33.00,NaN,4.86,...,NaN,NaN,NaN,NaN,3.0,29.5,Miami (FL),NaN,Jerome Brown,1987.0
145,1987,Rich Gannon,Delaware,QB,74.30,193,9.50,30.00,NaN,4.58,...,21.0,2.0,-3.0,0.0,NaN,NaN,Delaware,NaN,Rich Gannon,1987.0
146,1987,Christian Okoye,Azusa Pacific (CA),RB,73.60,250,9.75,33.00,NaN,NaN,...,40.0,42.0,294.0,0.0,NaN,NaN,Azusa Pacific,NaN,Christian Okoye,1987.0


In [10]:
df_College = df_NFLChampsMerged[['College']].copy()

In [11]:
df_College = df_College.drop_duplicates()
df_College.reset_index(drop=True,inplace=True)
df_College.reset_index(inplace=True)
df_College.rename(columns={"index":"collegeid"},inplace=True)
df_College

,collegeid,College
0,0,Alabama
1,1,Georgia
2,2,Northern Iowa
3,3,Kansas State
4,4,Pittsburgh
...,...,...
77,77,Kentucky
78,78,South Carolina
79,79,North Carolina
80,80,Delaware


In [12]:
df_Players = df_NFLChampsMerged[["Player_Id","Name","College","POS"]].copy()

In [13]:
df_Players = pd.merge(df_Players,df_College,on="College")
df_Players.drop(columns="College",inplace=True)
df_Players.reset_index(drop=True,inplace=True)
df_Players.rename(columns={"Player_Id": "playerid", "Name":"name", "POS":"position"}, inplace=True)
df_Players = df_Players[["playerid","name","collegeid","position"]]
df_Players

,playerid,name,collegeid,position
0,CollLa00,Landon Collins,0,SS
1,McClLe00,Le'Ron McClain,0,FB
2,ThomDe01,Derrick Thomas,0,OLB
3,GurlTo01,Todd Gurley,1,RB
4,WalsBl00,Blair Walsh,1,K
...,...,...,...,...
139,DawsDe00,Dermontti Dawson,77,C
140,SharSt00,Sterling Sharpe,78,WR
141,BartHa00,Harris Barton,79,C
142,GannRi00,Rich Gannon,80,QB


In [14]:
df_Draft =  df_NFLChampsMerged[["Player_Id",'Year', 'Rnd', 'Pick', 'Tm']].copy()
df_Draft.rename(columns={"Player_Id":"playerid","Year":"year","Rnd":"round","Pick":"pick","Tm":"nfl_teamid"},inplace=True)
df_Draft

,playerid,year,round,pick,nfl_teamid
0,CollLa00,2015,2,33,NYG
1,GurlTo01,2015,1,10,STL
2,JohnDa08,2015,3,86,ARI
3,LockTy00,2015,3,69,SEA
4,DonaAa00,2014,1,13,STL
...,...,...,...,...,...
143,BartHa00,1987,1,22,SFO
144,BrowJe00,1987,1,9,PHI
145,GannRi00,1987,4,98,NWE
146,OkoyCh00,1987,2,35,KAN


In [15]:
df_PlayerStats = df_NFLChampsMerged[["Player_Id","First4AV","Age", "To","AP1", "PB","St", "CarAV", "DrAV","G", "Cmp", "Pass_Att", "Pass_Yds", "Pass_TD","Pass_Int", "Rush_Att","Rush_Yds", "Rush_TDs", "Rec", "Rec_Yds", "Rec_Tds","Tkl", "Def_Int", "Sk"]].copy()
df_PlayerStats.rename(columns={"Player_Id":"playerid","First4AV":"first4avg","Age":"age","To":"to_year","AP1":"ap1","PB":"pb",\
                               "St":"st","CarAV":"career_avg","DrAV":"dr_avg","G":"games_count","Cmp":"completion",\
                               "Pass_Att":"pass_att","Pass_Yds":"pass_yds","Pass_TD":"pass_td","Pass_Int":"pass_int","Rush_Att":"rush_att",\
                               "Rush_Yds":"rush_yds","Rush_TDs":"rush_tds","Rec":"rec","Rec_Yds":"rec_yds","Rec_Tds":"rec_tds",\
                               "Tkl":"tackle","Def_Int":"def_int","Sk":"sk"},inplace=True)

df_PlayerStats['sk'] = pd.to_numeric(df_PlayerStats['sk'], errors='coerce')
df_PlayerStats = df_PlayerStats.fillna(0)
df_PlayerStats

,playerid,first4avg,age,to_year,ap1,pb,st,career_avg,dr_avg,games_count,...,pass_int,rush_att,rush_yds,rush_tds,rec,rec_yds,rec_tds,tackle,def_int,sk
0,CollLa00,5,21.0,2016.0,0,0,2,5.0,5.0,26.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146.0,6.0,3.0
1,GurlTo01,8,21.0,2016.0,0,1,2,8.0,8.0,23.0,...,0.0,416.0,1697.0,14.0,47.0,390.0,0.0,0.0,0.0,0.0
2,JohnDa08,9,23.0,2016.0,0,0,1,9.0,9.0,26.0,...,0.0,322.0,1444.0,18.0,83.0,967.0,6.0,0.0,0.0,0.0
3,LockTy00,9,23.0,2016.0,1,1,0,9.0,9.0,26.0,...,0.0,7.0,35.0,0.0,74.0,989.0,6.0,0.0,0.0,0.0
4,DonaAa00,31,23.0,2016.0,1,2,3,30.0,30.0,42.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,0.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,BartHa00,41,23.0,1996.0,2,1,10,78.0,78.0,138.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144,BrowJe00,52,22.0,1991.0,2,2,5,48.0,48.0,76.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,29.5,0.0
145,GannRi00,18,21.0,2004.0,2,4,8,99.0,0.0,157.0,...,104.0,521.0,2449.0,21.0,2.0,-3.0,0.0,0.0,0.0,0.0
146,OkoyCh00,32,26.0,1992.0,1,2,6,33.0,33.0,79.0,...,0.0,1246.0,4897.0,40.0,42.0,294.0,0.0,0.0,0.0,0.0


In [16]:
#df_CombineData = df_NFLChampsMerged[['Player_Id','Height (in)', 'Weight (lbs)','Hand Size (in)', 'Arm Length (in)', 'Wonderlic', '40 Yard','Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle', '3Cone','60Yd Shuttle']].copy()

df_CombineData = df_NFLChampsMerged.iloc[:,4:17].copy()
df_CombineData.rename(columns={df_CombineData.columns[0]: "height",df_CombineData.columns[1]: "weight",\
                               df_CombineData.columns[2]: "handsize",df_CombineData.columns[3]: "arm_length",\
                               df_CombineData.columns[4]: "wonderlic",df_CombineData.columns[5]: "forty_yard",\
                               df_CombineData.columns[6]: "bench_press",df_CombineData.columns[7]: "vert_leap",\
                               df_CombineData.columns[8]: "broad_jump",df_CombineData.columns[9]: "shuttle",\
                               df_CombineData.columns[10]: "threecone", df_CombineData.columns[11]: "sixty_yd_shuttle",\
                               df_CombineData.columns[12]: "playerid"},inplace=True)
df_CombineData = df_CombineData[['playerid', 'height', 'weight', 'handsize', 'arm_length', 'wonderlic', 'forty_yard',
       'bench_press', 'vert_leap', 'broad_jump', 'shuttle', 'threecone',
       'sixty_yd_shuttle']]
df_CombineData = df_CombineData.fillna(0)
df_CombineData

,playerid,height,weight,handsize,arm_length,wonderlic,forty_yard,bench_press,vert_leap,broad_jump,shuttle,threecone,sixty_yd_shuttle
0,CollLa00,72.00,228,9.38,31.50,17.0,4.53,0.0,35.0,120.0,4.33,7.38,11.94
1,GurlTo01,72.63,222,10.00,31.50,12.0,4.52,17.0,0.0,0.0,0.00,0.00,0.00
2,JohnDa08,72.63,224,9.63,31.25,0.0,4.50,25.0,41.5,127.0,4.27,6.82,0.00
3,LockTy00,69.88,182,8.38,30.00,0.0,4.40,0.0,35.5,121.0,4.07,6.89,11.14
4,DonaAa00,72.75,285,9.88,32.63,0.0,4.68,35.0,32.0,115.0,4.39,7.11,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,BartHa00,76.30,280,9.50,31.75,0.0,5.03,24.0,31.0,106.0,4.46,0.00,0.00
144,BrowJe00,74.30,292,9.75,33.00,0.0,4.86,17.0,0.0,0.0,0.00,0.00,0.00
145,GannRi00,74.30,193,9.50,30.00,0.0,4.58,7.0,32.5,110.0,4.11,0.00,0.00
146,OkoyCh00,73.60,250,9.75,33.00,0.0,0.00,21.0,31.0,0.0,0.00,0.00,0.00


In [17]:
df_College.rename(columns={"College":"college"},inplace=True)

In [18]:
#rds_connection_string = "postgres:'Grp#7_nflDB'@34.86.235.43:10801/nfl-da-grp7"
rds_connection_string = "postgres:cyclone1@localhost:5432/nfl-da-grp7"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [19]:
# Place holder for inserting Dataframe to postgres table

df_College.to_sql(name='college', con=engine, if_exists='append', index=False)
df_Players.to_sql(name='player', con=engine, if_exists='append', index=False)
df_Draft.to_sql(name='draft', con=engine, if_exists='append', index=False)
df_PlayerStats.to_sql(name='player_stats', con=engine, if_exists='append', index=False)
df_CombineData.to_sql(name='combine_data', con=engine, if_exists='append', index=False)
